In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import os
import sys
sys.path.insert(0, '../../src/')
import data_loader
pd.set_option('display.max_columns', None)

In [2]:
season_2018 = data_loader.load_data(seasons=["2018-19"])
seasons_ldg = data_loader.load_game_data(seasons=["2018-19"])

Loaded PBP-data


In [3]:
season_2018

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,PERSON2TYPE,PERSON3TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,PLAYER1_TEAM_CITY,PLAYER1_TEAM_ID,PLAYER1_TEAM_NICKNAME,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ABBREVIATION,PLAYER2_TEAM_CITY,PLAYER2_TEAM_ID,PLAYER2_TEAM_NICKNAME,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,PLAYER3_TEAM_CITY,PLAYER3_TEAM_ID,PLAYER3_TEAM_NICKNAME,SCORE,SCOREMARGIN,VISITORDESCRIPTION,WCTIMESTRING,season_name
0,0,0,PERIOD_BEGIN,2,21800794,NaN,12:00,1,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9:11 PM,2018-19
1,1,0,JUMP_BALL,4,21800794,Jump Ball Ayton vs. Faried: Tip to Okobo,12:00,1,HOME_PLAYER,VISITOR_PLAYER,HOME_PLAYER,1629028,Deandre Ayton,PHX,Phoenix,1610612756.0,Suns,202702,Kenneth Faried,HOU,Houston,1610612745.0,Rockets,1629059,Elie Okobo,PHX,Phoenix,1610612756.0,Suns,NaN,NaN,NaN,9:11 PM,2018-19
2,2,1,TURNOVER,7,21800794,Jackson Bad Pass Turnover (P1.T1),11:39,1,HOME_PLAYER,VISITOR_PLAYER,0,1628367,Josh Jackson,PHX,Phoenix,1610612756.0,Suns,201935,James Harden,HOU,Houston,1610612745.0,Rockets,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Harden STEAL (1 STL),9:11 PM,2018-19
3,3,Driving Layup,FIELD_GOAL_MADE,9,21800794,NaN,11:31,1,VISITOR_PLAYER,0,0,201935,James Harden,HOU,Houston,1610612745.0,Rockets,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2 - 0,-2,Harden 3' Driving Layup (2 PTS),9:12 PM,2018-19
4,4,2,FOUL,10,21800794,Bridges S.FOUL (P1.T1) (E.Dalen),11:31,1,HOME_PLAYER,VISITOR_PLAYER,TIMEOUT,1628969,Mikal Bridges,PHX,Phoenix,1610612756.0,Suns,201935,James Harden,HOU,Houston,1610612745.0,Rockets,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9:12 PM,2018-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582463,431,1,TIMEOUT,634,21801037,NaN,0:04,4,VISITOR_TEAM,0,0,1610612739,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cavaliers Timeout: Regular (Reg.6 Short 0),10:49 PM,2018-19
582464,432,0,SUBSTITUTION,635,21801037,NaN,0:04,4,VISITOR_PLAYER,VISITOR_PLAYER,0,1627790,Ante Zizic,CLE,Cleveland,1610612739.0,Cavaliers,203903,Jordan Clarkson,CLE,Cleveland,1610612739.0,Cavaliers,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUB: Clarkson FOR Zizic,10:50 PM,2018-19
582465,433,Fadeaway Jumper,FIELD_GOAL_MISSED,638,21801037,NaN,0:02,4,VISITOR_PLAYER,0,0,203917,Nik Stauskas,CLE,Cleveland,1610612739.0,Cavaliers,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MISS Stauskas 24' 3PT Fadeaway Jumper,10:51 PM,2018-19
582466,434,live,REBOUND,639,21801037,Kleber REBOUND (Off:5 Def:7),0:02,4,HOME_PLAYER,0,0,1628467,Maxi Kleber,DAL,Dallas,1610612742.0,Mavericks,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:51 PM,2018-19


### TODO => TESTING - STARTING LINEUPS

In [4]:
# still in development phase
def starters():
    GAME_ID_STR = "GAME_ID"

    # TODO -> load season_data + load game_data
    # TODO
    pbp_grouped = season_2018.groupby(GAME_ID_STR)
    game_data = pbp_grouped.size().to_frame(name="play_count")
    # TODO


    # assign ID to home and away team
    game_data["HOME TEAM ID"] = pbp_grouped.apply(lambda x: x[x["PERSON1TYPE"] == 'HOME_PLAYER']["PLAYER1_TEAM_ID"].iloc[0])
    game_data["VISITOR TEAM ID"] = pbp_grouped.apply(lambda x: x[x["PERSON1TYPE"] == 'VISITOR_PLAYER']["PLAYER1_TEAM_ID"].iloc[0])
    
    # iterrate though all matches in season
    for i, row in game_data.iterrows():
        # game_id = i
        home_team_id = row["HOME TEAM ID"]
        visitor_team_id = row["VISITOR TEAM ID"]
        
        home_players = season_2018[(season_2018["EVENTMSGTYPE"] == "SUBSTITUTION") & (season_2018["GAME_ID"] == i) & (season_2018["PERIOD"] == 1) &(season_2018["PLAYER1_TEAM_ID"] == home_team_id)]["PLAYER1_NAME"].unique()
        visitor_players = season_2018[(season_2018["EVENTMSGTYPE"] == "SUBSTITUTION") & (season_2018["GAME_ID"] == i) & (season_2018["PERIOD"] == 1) &(season_2018["PLAYER1_TEAM_ID"] == visitor_team_id)]["PLAYER1_NAME"].unique()

        # replace missing starters with 0
        if len(home_players) < 5:
            home_players.resize(5)
        if len(visitor_players) < 5:
            visitor_players.resize(5)
        
        game_data.at[i, "HOME_PLAYER_1" :] = home_players
        game_data.at[i, "VISITOR_PLAYER_1" :] = visitor_players

    print("DONE")




In [5]:
GAME_ID_STR = "GAME_ID"
pbp_grouped = season_2018.groupby(GAME_ID_STR)
game_data = pbp_grouped.size().to_frame(name="play_count")

In [6]:
game_data["HOME TEAM ID"] = pbp_grouped.apply(lambda x: x[x["PERSON1TYPE"] == 'HOME_PLAYER']["PLAYER1_TEAM_ID"].iloc[0])
game_data["VISITOR TEAM ID"] = pbp_grouped.apply(lambda x: x[x["PERSON1TYPE"] == 'VISITOR_PLAYER']["PLAYER1_TEAM_ID"].iloc[0])

### TESTING - STARTING LINEUPS

In [99]:
game_data

,play_count,HOME TEAM ID,VISITOR TEAM ID,HOME_PLAYER_1,HOME_PLAYER_2,HOME_PLAYER_3,HOME_PLAYER_4,HOME_PLAYER_5,VISITOR_PLAYER_1,VISITOR_PLAYER_2,VISITOR_PLAYER_3,VISITOR_PLAYER_4,VISITOR_PLAYER_5
GAME_ID,,,,,,,,,,,,,
21800001,492,1610612738.0,1610612755.0,Jayson Tatum,Gordon Hayward,Kyrie Irving,Jaylen Brown,Al Horford,Dario Saric,Joel Embiid,Robert Covington,Amir Johnson,Markelle Fultz
21800002,522,1610612744.0,1610612760.0,Damian Jones,Draymond Green,Kevin Durant,Klay Thompson,Stephen Curry,Dennis Schroder,Patrick Patterson,Paul George,Steven Adams,Terrance Ferguson
21800003,487,1610612766.0,1610612749.0,Marvin Williams,Cody Zeller,Jeremy Lamb,Nicolas Batum,Kemba Walker,Giannis Antetokounmpo,Malcolm Brogdon,Brook Lopez,Eric Bledsoe,Khris Middleton
21800004,478,1610612765.0,1610612751.0,Andre Drummond,Blake Griffin,Bruce Brown,Luke Kennard,Reggie Jackson,Jared Dudley,Joe Harris,Jarrett Allen,D'Angelo Russell,Caris LeVert
21800005,453,1610612754.0,1610612763.0,Thaddeus Young,Bojan Bogdanovic,Darren Collison,Myles Turner,TJ Leaf,JaMychal Green,Chandler Parsons,Mike Conley,Garrett Temple,Marc Gasol
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21801226,474,1610612749.0,1610612760.0,Bonzie Colson,D.J. Wilson,Ersan Ilyasova,Khris Middleton,Pat Connaughton,Dennis Schroder,Jerami Grant,Russell Westbrook,Steven Adams,Terrance Ferguson
21801227,443,1610612759.0,1610612742.0,Jakob Poeltl,Derrick White,Bryn Forbes,DeMar DeRozan,LaMarcus Aldridge,Dirk Nowitzki,Dwight Powell,Jalen Brunson,Justin Jackson,Devin Harris
21801228,434,1610612743.0,1610612750.0,Jamal Murray,Paul Millsap,Gary Harris,Nikola Jokic,Will Barton,Dario Saric,Josh Okogie,Tyus Jones,Gorgui Dieng,Andrew Wiggins


In [98]:
game_data[["HOME_PLAYER_1", "HOME_PLAYER_2", "HOME_PLAYER_3", "HOME_PLAYER_4", "HOME_PLAYER_5", "VISITOR_PLAYER_1", "VISITOR_PLAYER_2", "VISITOR_PLAYER_3", "VISITOR_PLAYER_4", "VISITOR_PLAYER_5"]] = np.nan

for i, row in game_data.iterrows():
    # game_id = i
    home_team_id = row["HOME TEAM ID"]
    visitor_team_id = row["VISITOR TEAM ID"]
    
    home_players = season_2018[(season_2018["EVENTMSGTYPE"] == "SUBSTITUTION") & (season_2018["GAME_ID"] == i) & (season_2018["PERIOD"] == 1) &(season_2018["PLAYER1_TEAM_ID"] == home_team_id)]["PLAYER1_NAME"].dropna().unique()
    visitor_players = season_2018[(season_2018["EVENTMSGTYPE"] == "SUBSTITUTION") & (season_2018["GAME_ID"] == i) & (season_2018["PERIOD"] == 1) &(season_2018["PLAYER1_TEAM_ID"] == visitor_team_id)]["PLAYER1_NAME"].dropna().unique()

    if len(home_players) < 5:
        players_1 = season_2018[(season_2018["GAME_ID"] == i) & (season_2018["PERIOD"] == 1) & (season_2018["PLAYER1_TEAM_ID"] == home_team_id)]["PLAYER1_NAME"].dropna().unique()
        players_2 = season_2018[(season_2018["GAME_ID"] == i) & (season_2018["PERIOD"] == 1) & (season_2018["PLAYER2_TEAM_ID"] == home_team_id)]["PLAYER2_NAME"].dropna().unique()
        players_3 = season_2018[(season_2018["GAME_ID"] == i) & (season_2018["PERIOD"] == 1) & (season_2018["PLAYER3_TEAM_ID"] == home_team_id)]["PLAYER3_NAME"].dropna().unique()
        out_players = season_2018[(season_2018["EVENTMSGTYPE"] == "SUBSTITUTION") & (season_2018["GAME_ID"] == i) & (season_2018["PERIOD"] == 1) &(season_2018["PLAYER1_TEAM_ID"] == home_team_id)]["PLAYER1_NAME"].dropna().unique()
        in_players = season_2018[(season_2018["EVENTMSGTYPE"] == "SUBSTITUTION") & (season_2018["GAME_ID"] == i) & (season_2018["PERIOD"] == 1) &(season_2018["PLAYER2_TEAM_ID"] == home_team_id)]["PLAYER2_NAME"].dropna().unique()

        lineup = reduce(np.union1d, (players_1, players_2, players_3, home_players))
        lineup = np.setdiff1d(lineup, in_players)
        lineup = np.setdiff1d(lineup, out_players)
        home_players = np.union1d(lineup, out_players)
    
    if len(visitor_players) < 5:
        players_1 = season_2018[(season_2018["GAME_ID"] == i) & (season_2018["PERIOD"] == 1) & (season_2018["PLAYER1_TEAM_ID"] == visitor_team_id)]["PLAYER1_NAME"].dropna().unique()
        players_2 = season_2018[(season_2018["GAME_ID"] == i) & (season_2018["PERIOD"] == 1) & (season_2018["PLAYER2_TEAM_ID"] == visitor_team_id)]["PLAYER2_NAME"].dropna().unique()
        players_3 = season_2018[(season_2018["GAME_ID"] == i) & (season_2018["PERIOD"] == 1) & (season_2018["PLAYER3_TEAM_ID"] == visitor_team_id)]["PLAYER3_NAME"].dropna().unique()
        out_players = season_2018[(season_2018["EVENTMSGTYPE"] == "SUBSTITUTION") & (season_2018["GAME_ID"] == i) & (season_2018["PERIOD"] == 1) &(season_2018["PLAYER1_TEAM_ID"] == visitor_team_id)]["PLAYER1_NAME"].dropna().unique()
        in_players = season_2018[(season_2018["EVENTMSGTYPE"] == "SUBSTITUTION") & (season_2018["GAME_ID"] == i) & (season_2018["PERIOD"] == 1) &(season_2018["PLAYER2_TEAM_ID"] == visitor_team_id)]["PLAYER2_NAME"].dropna().unique()

        lineup = reduce(np.union1d, (players_1, players_2, players_3, visitor_players))
        lineup = np.setdiff1d(lineup, in_players)
        lineup = np.setdiff1d(lineup, out_players)
        visitor_players = np.union1d(lineup, out_players)

    game_data.at[i, "HOME_PLAYER_1":"HOME_PLAYER_5"] = home_players[:5]
    game_data.at[i, "VISITOR_PLAYER_1" :] = visitor_players[:5]

print("DONE")

DONE


In [64]:
starter_5 = season_2018[(season_2018["GAME_ID"] == 21800002) & (season_2018["PERIOD"] == 1) & (season_2018["PLAYER1_TEAM_ID"] == "1610612744.0")]["PLAYER1_NAME"].dropna().unique()


In [82]:
players_1 = season_2018[(season_2018["GAME_ID"] == 21800002) & (season_2018["PERIOD"] == 1) & (season_2018["PLAYER1_TEAM_ID"] == "1610612744.0")]["PLAYER1_NAME"].dropna().unique()
players_2 = season_2018[(season_2018["GAME_ID"] == 21800002) & (season_2018["PERIOD"] == 1) & (season_2018["PLAYER2_TEAM_ID"] == "1610612744.0")]["PLAYER2_NAME"].dropna().unique()
players_3 = season_2018[(season_2018["GAME_ID"] == 21800002) & (season_2018["PERIOD"] == 1) & (season_2018["PLAYER3_TEAM_ID"] == "1610612744.0")]["PLAYER3_NAME"].dropna().unique()
out_players = season_2018[(season_2018["EVENTMSGTYPE"] == "SUBSTITUTION") & (season_2018["GAME_ID"] == 21800002) & (season_2018["PERIOD"] == 1) &(season_2018["PLAYER1_TEAM_ID"] == "1610612744.0")]["PLAYER1_NAME"].dropna().unique()
in_players = season_2018[(season_2018["EVENTMSGTYPE"] == "SUBSTITUTION") & (season_2018["GAME_ID"] == 21800002) & (season_2018["PERIOD"] == 1) &(season_2018["PLAYER2_TEAM_ID"] == "1610612744.0")]["PLAYER2_NAME"].dropna().unique()

In [83]:
players_1

array(['Damian Jones', 'Klay Thompson', 'Stephen Curry', 'Kevin Durant',
       'Draymond Green', 'Kevon Looney', 'Andre Iguodala'], dtype=object)

In [84]:
players_2

array(['Kevin Durant', 'Draymond Green', 'Stephen Curry', 'Kevon Looney',
       'Damian Jones', 'Andre Iguodala', 'Quinn Cook'], dtype=object)

In [85]:
players_3

array(['Draymond Green', 'Kevin Durant', 'Damian Jones', 'Kevon Looney'],
      dtype=object)

In [86]:
in_players

array(['Kevon Looney', 'Andre Iguodala', 'Draymond Green', 'Quinn Cook'],
      dtype=object)

In [87]:
out_players

array(['Draymond Green', 'Klay Thompson', 'Damian Jones', 'Stephen Curry'],
      dtype=object)

In [74]:
all_players = reduce(np.union1d, (players_1, players_2, players_3, out_players))
all_players

array(['Andre Iguodala', 'Damian Jones', 'Draymond Green', 'Kevin Durant',
       'Kevon Looney', 'Klay Thompson', 'Quinn Cook', 'Stephen Curry'],
      dtype=object)

In [88]:
lineup = np.setdiff1d(all_players, in_players)
lineup = np.setdiff1d(lineup, out_players)

In [89]:
lineup

array(['Kevin Durant'], dtype=object)

In [90]:
lineup = np.union1d(lineup, out_players)

In [91]:
out_players

array(['Draymond Green', 'Klay Thompson', 'Damian Jones', 'Stephen Curry'],
      dtype=object)

In [92]:
lineup

array(['Damian Jones', 'Draymond Green', 'Kevin Durant', 'Klay Thompson',
       'Stephen Curry'], dtype=object)